A notebook to compute precision and recall of an object detector. The notebook requires two files, detections.geojson and truths.geojson which contain the detection vectors and the reference vectors againgt which precision and recall are computed, respectively.

Precision is defined as the ratio TP/(TP+FP) and recall as the ratio TP/(TP+FN), where:

+ TP is the number of detection bounding boxes that intersect the ground-truth polygon for which the maximum intersection-over-union, calculated over each of the ground truth bounding boxes, is >= 0.5.
+ FP is the number of detection bounding boxes that intersect the ground-truth polygon for which the maximum intersection-over-union, calculated over each of the ground truth bounding boxes, is < 0.5.
+ FN is the number of ground-truth bounding boxes (boat bounding boxes which intersect the ground-truth polygon) for which the maximum intersection-over-union, calculated over each of the detection bounding boxes, is < 0.5.

Note that these definitions allow for multiple targets to be covered by the same detection bounding box (provided that the intersection-over-union is large enough).

We also introduce **area-weighted** precision and recall. The modification introduced compared to the previous definitions is that each true positive, false positive and false negative are weighted by the corresponding bounding box area. This is a means to assign importance to a correct/faulty detection or a miss based on size. 

In [ ]:
import json

from shapely.geometry import shape
from shapely.strtree import STRtree

# intersection-over-union threshold for valid detection
min_iou = 0.5

# open geojsons
detections = json.load(open('detections.geojson'))['features']
truths = json.load(open('truths.geojson'))['features']
    
# initialization
TP, FP, FN, wTP, wFP, wFN = 0, 0, 0, 0, 0, 0

detection_polys = [shape(detection['geometry']) for detection in detections]
truth_polys = [shape(truth['geometry']) for truth in truths]   

# create r-tree of truth polys
tree_truths = STRtree(truth_polys)

for detection_poly in detection_polys:

    detection_poly_area = detection_poly.area

    # find all truths which this detection_poly intersects
    results = tree_truths.query(detection_poly)

    # if there aren't any results, this is a false positive
    if not results:
        FP += 1
        wFP += detection_poly_area
    # if there are results, compute the maximum intersection over union over all results
    else:
        ious = []
        for result in results:
            iou = result.intersection(detection_poly).area / result.union(detection_poly).area
            ious.append(iou)
        if max(ious) >= min_iou:
            TP += 1 
            wTP += detection_poly_area
        else:
            FP += 1
            wFP += detection_poly_area

# create r-tree of detection polys
tree_detections = STRtree(detection_polys)

for truth_poly in truth_polys:

    truth_poly_area = truth_poly.area

    # find all detections which the truth poly intersects
    results = tree_detections.query(truth_poly)

    # if there aren't any results, this is a false negative
    if not results:
        FN += 1
        wFN += truth_poly_area
    # if there are results, compute the maximum intersection over union over all results
    else:
        ious = []
        for result in results:
            iou = result.intersection(truth_poly).area / result.union(truth_poly).area
            ious.append(iou)
        if max(ious) < min_iou:
            FN += 1 
            wFN += truth_poly_area            

# report metrics
print 'TP = {}, FP = {}, FN = {}, p = {}, r = {}'.format(TP, FP, FN, float(TP)/(TP + FP), float(TP)/(TP + FN))
print 'wTP = {}, wFP = {}, wFN = {}, wp = {}, wr = {}'.format(wTP, wFP, wFN, wTP/(wTP + wFP), wTP/(wTP + wFN))